<a href="https://colab.research.google.com/github/supercat-cpu/calc/blob/main/2023_%D0%92%D0%A8%D0%AD_%D0%94%D0%97_%E2%84%961_SPARK_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание № 1 (Apache Spark SQL)

**Цель**: Решить 5 задач на SQL с использованием базовых таблиц.

**Инструкции**:
0. Прочитать условия ДЗ
1. Выполнить задание на SQL;
2. Ознакомиться с документацией по SQL: [Ссылка](https://spark.apache.org/docs/latest/sql-ref.html);
3. Создать ноутбук с решениями задач (создайте копию на основе этого шаблона);
4. Каждый запрос должен содержать обязательные комментарии с объяснением логического смысла решения;
5. Скачать ноутбук в формате .ipynb и отправить на адрес электронной почты: ilya+hse@aniskovets.com.

**Оценка и дедлайн**:
- Максимальная оценка - 5 баллов;
- Дедлайн: 24.05.2023 23:59;
- Оценки будут опубликованы после сдачи **всех двух** домашних заданий.

**Таблицы**:
Для выполнения задания предполагается использование следующих таблиц, загруженных в систему в формате CSV с типами полей STRING:

- title_basics_csv
- title_principals_csv
- title_crew_csv
- title_episode_csv
- title_ratings_csv
- title_akas_csv
- name_basics_csv

**Преобразование типов**:
Для преобразования STRING в другой тип используйте конструкцию: 
```sql
CAST (column_name AS TYPE) AS column_name
```

**Пример**
```sql
CREATE TABLE test_table USING PARQUET AS 
  SELECT 
    CAST(averageRating AS decimal(2,1))  AS averageRating 
  FROM test_table_csv 
```

**Все подготовительные этапы по преобразованию типов нужно делать в разделе Initialization. После чего, рекомендуется использовать в запросах таблицы в формате PARQUET**

Перед этим не мешает проверить, что значения влезают в размерности этого типа
Список типов SQL: https://spark.apache.org/docs/latest/sql-ref-datatypes.html 

Домашнее задание необходимо сделать в виде ноутбука. Ноутбук должен запускаться без ошибок.

Во всех задачах ответом должно быть только ОДНО значение (число или строка).

Неправильный ответ - 0 баллов за задачу.
Правильный ответ на все тесты 1 балл за задачу, иначе пропорционально количеству пройденных тестов (количество тестов и какие именно заранее неизвестны).

**ВНИМАНИЕ! К каждому запросу необходимо писать комментарии. Комментарии должны обьяснять логический смысл решения. Отсутствие комментария к решению задачи штраф - оценка за задачу = оценка / 2!**

Всего 5 задач и 5 баллов

**В режиме проверки SQL запросы будут запускаться с любыми параметрами, отличными от тех, что вы выбрали.**

Скрипты не должны зависеть от выбранного вами параметра, а также от регистра строки. Все параметры должны быть изменяемыми (смотрите, пример в Вопросе № 0)

**Тестирование будет проводится с любым значением параметров, в том числе с отсутвующим в датасете, запрос должен выводить правильный результат!**

SQL запрос в ответе всегда должен быть один и параметризирован. Если необходимо пользуйтесь конструкцией WITH name AS () -  пример ниже

После завершения работы нажимайте в меню File/Download/Download ipnb, скачивайте файл и присылайте почтой на ilya+hse@aniskovets.com

Если заметили опечатки или появились вопросы пишите в телеграм: @aigmx



In [ ]:
!pip install pyspark===3.4.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!curl -O https://mars.ru77.ru/data/title.basics.tsv.gz
!curl -O https://mars.ru77.ru/data/title.crew.tsv.gz
!curl -O https://mars.ru77.ru/data/title.episode.tsv.gz
!curl -O https://mars.ru77.ru/data/title.principals.tsv.gz
!curl -O https://mars.ru77.ru/data/title.ratings.tsv.gz
!curl -O https://mars.ru77.ru/data/title.akas.tsv.gz
!curl -O https://mars.ru77.ru/data/name.basics.tsv.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  132M  100  132M    0     0  8615k      0  0:00:15  0:00:15 --:--:-- 11.8M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 51.7M  100 51.7M    0     0  6009k      0  0:00:08  0:00:08 --:--:-- 10.8M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 29.6M  100 29.6M    0     0  4788k      0  0:00:06  0:00:06 --:--:-- 6536k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  342M  100  342M    0     0  10.4M      0  0:00:32  0:00:32 --:--:-- 11.9M
  % Total    % Received % Xferd  Average Speed   Tim

In [ ]:
from pyspark.sql import SparkSession, SQLContext
from pyspark import SparkConf, SparkContext


spark = SparkSession.builder.master("local[2]").config("spark.driver.memory", "8g").appName("vse").enableHiveSupport().getOrCreate()
sql = spark.sql

In [ ]:

title_basics_csv = spark.read.csv("title.basics.tsv.gz", sep='\\t', nullValue='\\N', header=True, quote="", escape="")
title_basics_csv.createOrReplaceTempView("title_basics_csv")


title_principals_csv = spark.read.csv("title.principals.tsv.gz", sep='\\t', nullValue='\\N', header=True, quote="", escape="")
title_principals_csv.createOrReplaceTempView("title_principals_csv")


title_crew_csv = spark.read.csv("title.crew.tsv.gz", sep='\\t', nullValue='\\N', header=True, quote="", escape="")
title_crew_csv.createOrReplaceTempView("title_crew_csv")


title_episode_csv = spark.read.csv("title.episode.tsv.gz", sep='\\t', nullValue='\\N', header=True, quote="", escape="")
title_episode_csv.createOrReplaceTempView("title_episode_csv")


title_ratings_csv = spark.read.csv("title.ratings.tsv.gz", sep='\\t', nullValue='\\N', header=True, quote="", escape="")
title_ratings_csv.createOrReplaceTempView("title_ratings_csv")


title_akas_csv = spark.read.csv("title.akas.tsv.gz", sep='\\t', nullValue='\\N', header=True, quote="", escape="")
title_akas_csv.createOrReplaceTempView("title_akas_csv")


name_basics_csv = spark.read.csv("name.basics.tsv.gz", sep='\\t', nullValue='\\N', header=True, quote="", escape="")
name_basics_csv.createOrReplaceTempView("name_basics_csv")

In [ ]:
# Initialization
_ = sql("""DROP TABLE IF EXISTS title_basics""").collect()
_ = sql("""CREATE TABLE title_basics STORED AS PARQUET SELECT * FROM title_basics_csv""").collect()

_ = sql("""DROP TABLE IF EXISTS title_principals""").collect()
_ = sql("""CREATE TABLE title_principals STORED AS PARQUET SELECT * FROM title_principals_csv""").collect()

_ = sql("""DROP TABLE IF EXISTS title_crew""").collect()
_ = sql("""CREATE TABLE title_crew STORED AS PARQUET SELECT * FROM title_crew_csv""").collect()

_ = sql("""DROP TABLE IF EXISTS title_episode""").collect()
_ = sql("""CREATE TABLE title_episode STORED AS PARQUET SELECT * FROM title_episode_csv""").collect()

_ = sql("""DROP TABLE IF EXISTS title_ratings""").collect()
_ = sql("""CREATE TABLE title_ratings STORED AS PARQUET SELECT * FROM title_ratings_csv""").collect()

_ = sql("""DROP TABLE IF EXISTS title_akas""").collect()
_ = sql("""CREATE TABLE title_akas STORED AS PARQUET SELECT * FROM title_akas_csv""").collect()

_ = sql("""DROP TABLE IF EXISTS name_basics""").collect()
_ = sql("""CREATE TABLE name_basics STORED AS PARQUET SELECT * FROM name_basics_csv""").collect()

In [ ]:
sql("DESCRIBE FORMATTED title_basics").show(50, truncate=False)
sql("DESCRIBE FORMATTED title_principals").show(50, truncate=False)
sql("DESCRIBE FORMATTED title_crew").show(50, truncate=False)
sql("DESCRIBE FORMATTED title_episode").show(50, truncate=False)
sql("DESCRIBE FORMATTED title_ratings").show(50, truncate=False)
sql("DESCRIBE FORMATTED title_akas").show(50, truncate=False)
sql("DESCRIBE FORMATTED name_basics").show(50, truncate=False)

+----------------------------+--------------------------------------------------------------+-------+
|col_name                    |data_type                                                     |comment|
+----------------------------+--------------------------------------------------------------+-------+
|tconst                      |string                                                        |null   |
|titleType                   |string                                                        |null   |
|primaryTitle                |string                                                        |null   |
|originalTitle               |string                                                        |null   |
|isAdult                     |string                                                        |null   |
|startYear                   |string                                                        |null   |
|endYear                     |string                                              

Вопрос №0 (не оценивается, показан, как пример). Сколько произведений имеют средний рейтинг выше заданного {threshold}?

In [ ]:
query_0 = """
/*
  Чтобы посчитать количество произведений находим все рейтинги больше, чем заданный порог.
  Так как надо найти все произведения, а не только фильмы, нам не нужно делать фильтр по типу произведения titleType
*/
SELECT 
  COUNT(tconst) as cnt_above_threshold
FROM  title_ratings
WHERE averageRating > {threshold};
"""
sql(query_0, threshold=5.0).show(truncate=False)

+----------------------+
|movies_above_threshold|
+----------------------+
|1029097               |
+----------------------+



Вопрос №1. Найдите среднюю продолжительность (title_basics:runtimeMinutes) фильмов (title_basics:titleType=movie) определенного жанра {genre}. 

Среднюю продолжительность надо рассчитать только для тех фильмов, где title_basics:runtimeMinutes не равен NULL. 

In [ ]:
query_1 = """
SELECT -1 AS answer -- ваш ответ тут 
"""
sql(query_1, genre='comedY').show(truncate=False)

+------+
|answer|
+------+
|-1    |
+------+



Вопрос №2. Сколько фильмов (title_basics:titleType=movie) определенного жанра (title_basics:genres) {genre} было выпущено в определенный год (title_basics:startYear) {year}?


In [ ]:
query_2 = """
SELECT -1 AS answer -- ваш ответ тут 
"""
sql(query_2, genre='Comedy', year=2022).show(truncate=False)

+------+
|answer|
+------+
|-1    |
+------+



Вопрос №3. Сколько фильмов определенного жанра (title_basics:genres) {genre}, в которых снялся конкретный актер (name_basics:primaryName) {actor}, имеют средний рейтинг (title_ratings:averageRating) выше заданного порога {threshold}?







In [ ]:
query_3 = """
SELECT -1 AS answer -- ваш ответ тут 
"""
sql(query_3, genre='comedy', actor='James Cagney', threshold=5.3).show(truncate=False)

+------+
|answer|
+------+
|-1    |
+------+



Вопрос №4. Сколько телесериалов (title_basics:titleType=tvSeries) в определенном жанре (title_basics:genres) {genre} имеют более определенного количества (title_episode:seasonNumber > {seasons})?


In [ ]:
query_4 = """
SELECT -1 AS answer -- ваш ответ тут 
"""
sql(query_4, genre='Comedy', seasons=7).show(truncate=False)

+------+
|answer|
+------+
|-1    |
+------+



Вопрос № 5. Сколько фильмов (title_basics:titleType=movie)  создал конкретный сценарист (title_crew:writers, name_basics:primaryName) {writer} в определенном жанре (title_basics:genres) {genre}?

In [ ]:
query_5  = """
SELECT 0 AS answer -- ваш ответ тут 
"""
sql(query_5, genre='comedy', writer='james cagney').show(truncate=False)

+------+
|answer|
+------+
|0     |
+------+



In [ ]:
# PLEASE DO NOT REMOVE THIS CELL

# Generate answers
for n in range(1, 6):
  print(f"-- answer {n}\n " + globals()[f"query_{n}"] + ";\n")

KeyError: ignored